In [2]:
import requests
import torch
from PIL import Image
from io import BytesIO

from llava.constants import IMAGE_TOKEN_INDEX
from llava.conversation import conv_templates
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, KeywordsStoppingCriteria

/Users/dog/.pyenv/versions/3.11.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/dog/.pyenv/versions/3.11.1/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/dog/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torch/distributed/_functional_collectives.py:22: UserWarning: Unable to import torchdynamo util `is_torchdynamo_compiling`, so won't support torchdynamo correctly
  warnings.warn(


ImportError: cannot import name 'LlavaLlamaForCausalLM' from 'llava.model' (/Users/dog/.pyenv/versions/3.11.1/lib/python3.11/site-packages/llava/model/__init__.py)

In [7]:
disable_torch_init()

model_path = "microsoft/llava-rad"
model_base = "lmsys/vicuna-7b-v1.5"
model_name = "llavarad"
conv_mode = "v1"

tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name)

ImportError: cannot import name 'MODEL_TYPE_TO_PEFT_MODEL_MAPPING' from 'peft.mapping' (/Users/dog/.pyenv/versions/3.11.1/lib/python3.11/site-packages/peft/mapping.py)

In [4]:
def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

In [5]:
image_file = "/Users/dog/Downloads/drive-download-20250427T052722Z-001/png/AB_training_13.png" # CXR w pneumothorax from Open-I
query = "<image>\nDescribe the findings of the chest x-ray.\n"
conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], query)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

image = load_image(image_file)
image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0].half().unsqueeze(0).cuda()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

stopping_criteria = KeywordsStoppingCriteria(["</s>"], tokenizer, input_ids)

with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=image_tensor,
        do_sample=False,
        temperature=9.0,
        max_new_tokens=5024,
        use_cache=True,
        stopping_criteria=[stopping_criteria])

outputs = tokenizer.batch_decode(output_ids[:, input_ids.shape[1]:], skip_special_tokens=True)[0]
outputs = outputs.strip()
print(outputs)

The lungs are clear.  Cardiomediastinal silhouette and hilar contours
 are unremarkable.  No pleural effusion or pneumothorax.
